In [1]:
import pickle as pkl
import pandas as pd
import torch
import numpy as np
import scipy.io
import h5py
import sklearn.metrics
import torch.nn as nn
from scipy.ndimage import gaussian_filter

In [84]:
cd /Users/abhay/GoogleDrive/BL

/Users/abhay/GoogleDrive/BL


In [85]:
pwd

'/Users/abhay/GoogleDrive/BL'

## Paper#1 data format

In [3]:
cd convdiff_2pi_n1500_t21_test/

/Users/abhay/GoogleDrive/BL/convdiff_2pi_n1500_t21_test


In [4]:
ls

t.pkl  u.pkl  x.pkl


In [9]:
tp = pkl.load(open("t.pkl",'rb'))

In [12]:
tp.shape

(50, 61)

In [13]:
type(tp)

numpy.ndarray

In [14]:
tp

array([[0.  , 0.01, 0.02, ..., 0.58, 0.59, 0.6 ],
       [0.  , 0.01, 0.02, ..., 0.58, 0.59, 0.6 ],
       [0.  , 0.01, 0.02, ..., 0.58, 0.59, 0.6 ],
       ...,
       [0.  , 0.01, 0.02, ..., 0.58, 0.59, 0.6 ],
       [0.  , 0.01, 0.02, ..., 0.58, 0.59, 0.6 ],
       [0.  , 0.01, 0.02, ..., 0.58, 0.59, 0.6 ]])

In [190]:
xp = pkl.load(open("/Users/abhay/GoogleDrive/BL/convdiff_2pi_n1500_t21_test/x.pkl",'rb'))

In [191]:
xp.shape

(50, 1500, 2)

In [192]:
xp

array([[[0.        , 0.10649467],
        [0.        , 0.        ],
        [0.53247333, 0.        ],
        ...,
        [0.44497349, 0.77177244],
        [5.98491456, 0.20316569],
        [5.16242981, 3.35059861]],

       [[0.        , 0.10649467],
        [0.        , 0.        ],
        [0.21298933, 0.        ],
        ...,
        [5.25641594, 3.6547031 ],
        [1.54490967, 4.01760941],
        [5.16242981, 3.35059861]],

       [[0.        , 0.10649467],
        [0.        , 0.        ],
        [0.10649467, 0.        ],
        ...,
        [4.02348354, 3.29442017],
        [4.10732479, 3.37426775],
        [5.16242981, 3.35059861]],

       ...,

       [[0.        , 0.        ],
        [0.10649467, 0.        ],
        [0.319484  , 0.        ],
        ...,
        [5.25641594, 3.6547031 ],
        [6.22393817, 0.37273133],
        [4.02348354, 3.29442017]],

       [[0.        , 0.        ],
        [0.42597866, 0.        ],
        [0.53247333, 0.        ],
        .

In [20]:
xp[2][700:710][:]

array([[1.20178045, 1.09023122],
       [2.52097748, 4.94480908],
       [1.19351624, 3.19421184],
       [4.86715052, 3.25166565],
       [5.41539998, 6.15877808],
       [2.06894375, 4.12649281],
       [4.8335521 , 1.80261011],
       [2.43174944, 2.07054467],
       [1.10341659, 3.52015333],
       [1.83998901, 5.78169072]])

In [21]:
up = pkl.load(open("u.pkl",'rb'))

In [23]:
up.shape

(50, 61, 1500, 1)

In [24]:
# NUM_simulations X Time_steps X Grid_points

In [214]:
up[1,1,:,:].shape

(1500, 1)

## Neural Operator paper data

In [29]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# reading data
class MatReader(object):
    def __init__(self, file_path, to_torch=True, to_cuda=False, to_float=True):
        super(MatReader, self).__init__()

        self.to_torch = to_torch
        self.to_cuda = to_cuda
        self.to_float = to_float

        self.file_path = file_path

        self.data = None
        self.old_mat = None
        self._load_file()

    def _load_file(self):
        try:
            self.data = scipy.io.loadmat(self.file_path)
            self.old_mat = True
        except:
            self.data = h5py.File(self.file_path)
            self.old_mat = False

    def load_file(self, file_path):
        self.file_path = file_path
        self._load_file()

    def read_field(self, field):
        x = self.data[field]

        if not self.old_mat:
            x = x[()]
            x = np.transpose(x, axes=range(len(x.shape) - 1, -1, -1))

        if self.to_float:
            x = x.astype(np.float32)

        if self.to_torch:
            x = torch.from_numpy(x)

            if self.to_cuda:
                x = x.cuda()

        return x

    def set_cuda(self, to_cuda):
        self.to_cuda = to_cuda

    def set_torch(self, to_torch):
        self.to_torch = to_torch

    def set_float(self, to_float):
        self.to_float = to_float


In [387]:
ls

2020.10.12.20211342.full.pdf       burgers_data_R10.mat
Burgers_R10.zip                    convdiff_2pi_n1500_t21_test/
COVID-19/                          covid_state_daily_test_u.pkl
Dec/                               covid_state_daily_train_u.pkl
NavierStokes_V1e-4_N10000_T30.zip  covid_state_train_x.pkl
NavierStokes_V1e-5_N1200_T20.mat   data_prep_covid.ipynb
NavierStokes_V1e-5_N1200_T20.zip   fourier_neural_operator/
Project/                           updated_covid_us.csv


In [37]:
################################################################
#  configurations
################################################################
ntrain = 1000
ntest = 100

sub = 1 #subsampling rate
h = 2**10 // sub
s = h

batch_size = 20
learning_rate = 0.001

epochs = 500
step_size = 100
gamma = 0.5

modes = 16
width = 64

In [388]:
################################################################
# read data
################################################################
#dataloader = MatReader('./burgers_data_R10.mat')
dataloader = MatReader('./NavierStokes_V1e-5_N1200_T20.mat')
x_data = dataloader.read_field('a')[:,::sub]
y_data = dataloader.read_field('u')[:,::sub]

x_train = x_data[:ntrain,:]
y_train = y_data[:ntrain,:]
x_test = x_data[-ntest:,:]
y_test = y_data[-ntest:,:]

# cat the locations information
grid = np.linspace(0, 2*np.pi, s).reshape(1, s, 1)
grid = torch.tensor(grid, dtype=torch.float)
x_train = torch.cat([x_train.reshape(ntrain,s,1), grid.repeat(ntrain,1,1)], dim=2)
x_test = torch.cat([x_test.reshape(ntest,s,1), grid.repeat(ntest,1,1)], dim=2)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

# model

RuntimeError: shape '[1000, 1024, 1]' is invalid for input of size 4096000

In [391]:
from timeit import default_timer
import scipy.io

torch.manual_seed(0)
np.random.seed(0)
################################################################
# configs
################################################################
TRAIN_PATH = 'NavierStokes_V1e-5_N1200_T20.mat'
TEST_PATH = 'NavierStokes_V1e-5_N1200_T20.mat'

ntrain = 1000
ntest = 200

modes = 12
width = 20

batch_size = 20
batch_size2 = batch_size

epochs = 500
learning_rate = 0.0025
scheduler_step = 100
scheduler_gamma = 0.5

print(epochs, learning_rate, scheduler_step, scheduler_gamma)

path = 'ns_fourier_2d_rnn_V10000_T20_N'+str(ntrain)+'_ep' + str(epochs) + '_m' + str(modes) + '_w' + str(width)
path_model = 'model/'+path
path_train_err = 'results/'+path+'train.txt'
path_test_err = 'results/'+path+'test.txt'
path_image = 'image/'+path

runtime = np.zeros(2, )
t1 = default_timer()

sub = 1
S = 64
T_in = 10
T = 10
step = 1

500 0.0025 100 0.5


In [392]:
reader = MatReader(TRAIN_PATH)
train_a = reader.read_field('u')[:ntrain,::sub,::sub,:T_in]
train_u = reader.read_field('u')[:ntrain,::sub,::sub,T_in:T+T_in]

reader = MatReader(TEST_PATH)
test_a = reader.read_field('u')[-ntest:,::sub,::sub,:T_in]
test_u = reader.read_field('u')[-ntest:,::sub,::sub,T_in:T+T_in]

print(train_u.shape)
print(test_u.shape)

torch.Size([1000, 64, 64, 10])
torch.Size([200, 64, 64, 10])


In [393]:
print(train_a.shape)
print(test_a.shape)

torch.Size([1000, 64, 64, 10])
torch.Size([200, 64, 64, 10])


In [41]:
x_data.shape

torch.Size([2048, 8192])

In [42]:
x_train.shape

torch.Size([1000, 8192])

In [44]:
grid.shape

torch.Size([1, 1024, 1])

In [46]:
x_test.shape

torch.Size([100, 8192])

In [47]:
dataloader1 = MatReader('./NavierStokes_V1e-5_N1200_T20.mat')
x_data1 = dataloader1.read_field('a')[:,::sub]
y_data1 = dataloader1.read_field('u')[:,::sub]

In [48]:
x_data1.shape

torch.Size([1200, 64, 64])

In [49]:
y_data1.shape

torch.Size([1200, 64, 64, 20])

In [54]:
max(y_data1[1,1,:,1])

tensor(0.4615)

In [55]:
ls

2020.10.12.20211342.full.pdf       NavierStokes_V1e-5_N1200_T20.zip
Burgers_R10.zip                    Project/
COVID-19/                          Untitled.ipynb
Dec/                               burgers_data_R10.mat
NavierStokes_V1e-4_N10000_T30.zip  convdiff_2pi_n1500_t21_test/
NavierStokes_V1e-5_N1200_T20.mat   fourier_neural_operator/


## READ COVID DATA 

### https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series

In [ ]:
#/Users/abhay/GoogleDrive/BL/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv

In [3]:
pwd

'/Users/abhay/GoogleDrive/BL'

In [4]:
cov_df = pd.read_csv("/Users/abhay/GoogleDrive/BL/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv")

In [5]:
len(cov_df)

3340

In [65]:
list(cov_df.columns) 

['UID',
 'iso2',
 'iso3',
 'code3',
 'FIPS',
 'Admin2',
 'Province_State',
 'Country_Region',
 'Lat',
 'Long_',
 'Combined_Key',
 '1/22/20',
 '1/23/20',
 '1/24/20',
 '1/25/20',
 '1/26/20',
 '1/27/20',
 '1/28/20',
 '1/29/20',
 '1/30/20',
 '1/31/20',
 '2/1/20',
 '2/2/20',
 '2/3/20',
 '2/4/20',
 '2/5/20',
 '2/6/20',
 '2/7/20',
 '2/8/20',
 '2/9/20',
 '2/10/20',
 '2/11/20',
 '2/12/20',
 '2/13/20',
 '2/14/20',
 '2/15/20',
 '2/16/20',
 '2/17/20',
 '2/18/20',
 '2/19/20',
 '2/20/20',
 '2/21/20',
 '2/22/20',
 '2/23/20',
 '2/24/20',
 '2/25/20',
 '2/26/20',
 '2/27/20',
 '2/28/20',
 '2/29/20',
 '3/1/20',
 '3/2/20',
 '3/3/20',
 '3/4/20',
 '3/5/20',
 '3/6/20',
 '3/7/20',
 '3/8/20',
 '3/9/20',
 '3/10/20',
 '3/11/20',
 '3/12/20',
 '3/13/20',
 '3/14/20',
 '3/15/20',
 '3/16/20',
 '3/17/20',
 '3/18/20',
 '3/19/20',
 '3/20/20',
 '3/21/20',
 '3/22/20',
 '3/23/20',
 '3/24/20',
 '3/25/20',
 '3/26/20',
 '3/27/20',
 '3/28/20',
 '3/29/20',
 '3/30/20',
 '3/31/20',
 '4/1/20',
 '4/2/20',
 '4/3/20',
 '4/4/20',
 '4/5

In [68]:
cov_df['Lat'].describe()

count    3340.000000
mean       36.707212
std         9.062922
min       -14.271000
25%        33.895587
50%        38.002344
75%        41.573069
max        69.314792
Name: Lat, dtype: float64

In [88]:
len(cov_df[(cov_df['Lat'] <= 0)])

109

In [95]:
cov_df[(cov_df['Lat'] > 0) & (cov_df['Lat'] < 17)]
#['Lat'].describe()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20
571,316,GU,GUM,316,66.0,NaN,Guam,US,13.4443,144.7937,...,7079,7137,7149,7168,7183,7193,7193,7193,7211,7238
2121,580,MP,MNP,580,69.0,NaN,Northern Mariana Islands,US,15.0979,145.6739,...,113,113,113,113,115,115,115,116,116,116


In [96]:
cov_df1 = cov_df[(cov_df['Lat'] > 17)]

In [98]:
cov_df1['Lat'].describe()

count    3229.000000
mean       37.964639
std         6.097623
min        17.982429
25%        34.356879
50%        38.214503
75%        41.686303
max        69.314792
Name: Lat, dtype: float64

In [111]:
cov_df1['Long_'].describe()

count    3229.000000
mean      -91.684503
std        13.410546
min      -174.159600
25%       -98.110839
50%       -89.950642
75%       -82.990997
max       -64.896300
Name: Long_, dtype: float64

In [110]:
cov_df[(cov_df['Long_'] < -160)]

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20
69,84002013,US,USA,840,2013.0,Aleutians East,Alaska,US,55.322224,-161.972202,...,40,40,40,40,40,40,40,40,40,40
70,84002016,US,USA,840,2016.0,Aleutians West,Alaska,US,52.323300,-174.159600,...,161,161,161,165,167,169,169,169,169,169
84,84002158,US,USA,840,2158.0,Kusilvak,Alaska,US,62.154292,-163.396788,...,626,643,651,662,669,670,672,672,686,688
86,84002180,US,USA,840,2180.0,Nome,Alaska,US,64.903207,-164.035380,...,254,256,257,262,264,264,264,264,265,265
100,16,AS,ASM,16,60.0,NaN,American Samoa,US,-14.271000,-170.132000,...,0,0,0,0,0,0,0,0,0,0


In [109]:
cov_df1.groupby('Province_State').sum()

,UID,code3,FIPS,Lat,Long_,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,...,12/13/20,12/14/20,12/15/20,12/16/20,12/17/20,12/18/20,12/19/20,12/20/20,12/21/20,12/22/20
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,5628071489,56280,71489.0,2203.246784,-5809.578199,0,0,0,0,0,...,295631,297895,301533,305640,310335,315683,319904,322452,324832,329811
Alaska,2436062340,24360,62340.0,1747.177765,-4293.070291,0,0,0,0,0,...,40933,41361,41643,42255,42641,43129,43462,43662,43819,44259
Arizona,1260060208,12600,60208.0,505.138555,-1671.948482,0,0,0,0,1,...,408442,420237,424371,429219,435036,442671,448231,453597,461345,467204
Arkansas,6300380625,63000,380625.0,2618.391704,-6932.548370,0,0,0,0,0,...,182606,183922,186117,188682,191757,194607,197289,198798,200302,202276
California,4872351364,48720,351364.0,2194.949775,-7002.258461,0,0,0,0,2,...,1585378,1611493,1644742,1708559,1753974,1802195,1842557,1884033,1923887,1958508
Colorado,5376515983,53760,515983.0,2491.870366,-6751.119357,0,0,0,0,0,...,288168,291079,293357,296691,300389,304082,306573,308865,311011,313527
Connecticut,672072064,6720,72064.0,332.909441,-581.254637,0,0,0,0,0,...,145706,152848,154650,157068,159455,162120,162120,162120,166687,168287
Delaware,252030009,2520,30009.0,117.327830,-226.599712,0,0,0,0,0,...,45364,46359,47039,47824,48650,48990,50016,50496,50937,51501
District of Columbia,84011001,840,11001.0,38.904178,-77.016560,0,0,0,0,0,...,24874,25038,25339,25602,25830,26104,26342,26601,26740,26900


In [112]:
cov_df1['Long_'].describe()

count    3229.000000
mean      -91.684503
std        13.410546
min      -174.159600
25%       -98.110839
50%       -89.950642
75%       -82.990997
max       -64.896300
Name: Long_, dtype: float64

In [117]:
date_num = list(range(1,337))

In [118]:
date_num

[1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 170,
 171,
 172,
 173,
 174,
 175,
 176,
 177,
 178,
 179,
 180,
 181,
 182,
 183,
 184,
 185

In [133]:
new_name = ['UID',
 'iso2',
 'iso3',
 'code3',
 'FIPS',
 'Admin2',
 'Province_State',
 'Country_Region',
 'Lat',
 'Long_',
 'Combined_Key']

In [130]:
new_name = new_name.append('5')

In [141]:
new_name_all = new_name

In [142]:
new_name_all

['UID',
 'iso2',
 'iso3',
 'code3',
 'FIPS',
 'Admin2',
 'Province_State',
 'Country_Region',
 'Lat',
 'Long_',
 'Combined_Key',
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 50,
 51,
 52,
 53,
 54,
 55,
 56,
 57,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 79,
 80,
 81,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100,
 101,
 102,
 103,
 104,
 105,
 106,
 107,
 108,
 109,
 110,
 111,
 112,
 113,
 114,
 115,
 116,
 117,
 118,
 119,
 120,
 121,
 122,
 123,
 124,
 125,
 126,
 127,
 128,
 129,
 130,
 131,
 132,
 133,
 134,
 135,
 136,
 137,
 138,
 139,
 140,
 141,
 142,
 143,
 144,
 145,
 146,
 147,
 148,
 149,
 150,
 151,
 152,
 153,
 154,
 155,
 156,
 157,
 158,
 159,
 160,
 161,
 162,
 163,
 

In [143]:
cov_df1.columns = new_name_all

In [144]:
cov_df1.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,327,328,329,330,331,332,333,334,335,336
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3300,3329,3426,3510,3570,3647,3698,3741,3780,3841
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,10806,10898,11061,11212,11364,11556,11722,11827,11952,12155
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1272,1275,1292,1296,1309,1318,1330,1336,1336,1363
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,1441,1455,1504,1520,1548,1577,1601,1613,1628,1660
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,3776,3803,3881,3950,4036,4118,4191,4218,4234,4313


In [145]:
len(cov_df1)

3229

In [154]:
cov_df1[1].describe()

count    3229.000000
mean        0.000310
std         0.017598
min         0.000000
25%         0.000000
50%         0.000000
75%         0.000000
max         1.000000
Name: 1, dtype: float64

In [156]:
def pickle_data(data_dict, path="./"):
    if not os.path.isdir(path):
        os.mkdir(path)

    for name, data in data_dict.items():
        with open(path+name+".pkl", "wb") as f:
            pkl.dump(data, f, protocol=4)

In [155]:
n_sim = 1

In [ ]:
t = np.zeros((n_sim, 180))
x = np.zeros((n_sim, *data_dicts[0]["x"].shape))
u = np.zeros((n_sim, *data_dicts[0]["u"].shape))

In [236]:
tp = np.array(range(1,181))

In [237]:
tp.reshape(1,180)

array([[  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178

In [163]:
mkdir covid_pkl

In [240]:
with open("covid_pkl_train_"+"t.pkl", "wb") as f:
    pkl.dump(tp.reshape(1,180), f, protocol=4)

In [241]:
tt = pkl.load(open("covid_pkl_train_t.pkl",'rb'))
tt

array([[  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,  13,
         14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,  26,
         27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,  39,
         40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,  52,
         53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,  65,
         66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,  78,
         79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,  91,
         92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103, 104,
        105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117,
        118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130,
        131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
        144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156,
        157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169,
        170, 171, 172, 173, 174, 175, 176, 177, 178

In [242]:
tpt = np.array(range(1,301)).reshape(1,300)
with open("covid_pkl_test_"+"t.pkl", "wb") as f:
    pkl.dump(tpt.reshape(1,300), f, protocol=4)

In [169]:
#x = np.zeros((n_sim, *data_dicts[0]["x"].shape))
xp = np.zeros((1, 3229, 2))

In [170]:
xp.shape

(1, 3229, 2)

In [176]:
cov_df1

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,327,328,329,330,331,332,333,334,335,336
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,3300,3329,3426,3510,3570,3647,3698,3741,3780,3841
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,10806,10898,11061,11212,11364,11556,11722,11827,11952,12155
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,1272,1275,1292,1296,1309,1318,1330,1336,1336,1363
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,1441,1455,1504,1520,1548,1577,1601,1613,1628,1660
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,3776,3803,3881,3950,4036,4118,4191,4218,4234,4313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3334,84056037,US,USA,840,56037.0,Sweetwater,Wyoming,US,41.659439,-108.882788,...,2451,2518,2544,2574,2649,2696,2702,2718,2762,2780
3335,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,-110.589080,...,1905,1930,1940,1958,1974,1990,1992,2002,2019,2033
3336,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,-110.547578,...,1307,1327,1352,1363,1372,1399,1406,1416,1446,1460
3338,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,-107.680187,...,639,644,659,662,665,690,695,699,713,720


In [206]:
i = 0
xp = np.zeros((1, 3229, 2))
x_list = [[]]
for index, row in cov_df1.iterrows():
    #print([row['Lat'], row['Long_']])
    x_list[0].append([row['Lat'], row['Long_']])
xp = np.asarray(x_list)

In [207]:
xp.shape

(1, 3229, 2)

In [208]:
xp

array([[[  32.53952745,  -86.64408227],
        [  30.72774991,  -87.72207058],
        [  31.868263  ,  -85.3871286 ],
        ...,
        [  41.2878183 , -110.5475782 ],
        [  43.90451606, -107.680187  ],
        [  43.83961191, -104.5674881 ]]])

In [209]:
with open("covid_pkl_train_"+"x.pkl", "wb") as f:
    pkl.dump(xp, f, protocol=4)

In [210]:
with open("covid_pkl_test_"+"x.pkl", "wb") as f:
    pkl.dump(xp, f, protocol=4)

In [200]:
x_list[0].append([1,2])
x_list[0].append([11,21])

### Cummulative cases 

In [222]:
u_list = []

In [223]:
for i in range(1,181):
    u_list.append(cov_df1[i].values)

In [231]:
a,b = np.asarray(u_list).shape
a,b

(180, 3229)

In [232]:
up = np.asarray(u_list).reshape(1,a,b,1)

In [233]:
up.shape

(1, 180, 3229, 1)

In [234]:
with open("covid_pkl_train_"+"u.pkl", "wb") as f:
    pkl.dump(up, f, protocol=4)

In [243]:
#prepare test data
u_list = []
for i in range(1,301):
    u_list.append(cov_df1[i].values)
a,b = np.asarray(u_list).shape
print(a,b)
upt = np.asarray(u_list).reshape(1,a,b,1)

300 3229


In [244]:
with open("covid_pkl_test_"+"u.pkl", "wb") as f:
    pkl.dump(upt, f, protocol=4)

## Daily cases

In [248]:
u_list = []
for i in range(1,181):
    u_list.append(cov_df1[i+1].values - cov_df1[i].values)
a,b = np.asarray(u_list).shape
print(a,b)
up_daily = np.asarray(u_list).reshape(1,a,b,1)
with open("covid_daily_train_"+"u.pkl", "wb") as f:
    pkl.dump(upt, f, protocol=4)

180 3229


Test Data

In [249]:
u_list = []
for i in range(1,301):
    u_list.append(cov_df1[i+1].values - cov_df1[i].values)
a,b = np.asarray(u_list).shape
print(a,b)
up_daily = np.asarray(u_list).reshape(1,a,b,1)
with open("covid_daily_test_"+"u.pkl", "wb") as f:
    pkl.dump(upt, f, protocol=4)

300 3229


## State wise data

In [251]:
cov_df1.to_csv("updated_covid_us.csv")

In [6]:
cov_df1 = pd.read_csv("updated_covid_us.csv")
cov_df1

,Unnamed: 0,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,...,327,328,329,330,331,332,333,334,335,336
0,0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,...,3300,3329,3426,3510,3570,3647,3698,3741,3780,3841
1,1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,...,10806,10898,11061,11212,11364,11556,11722,11827,11952,12155
2,2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,...,1272,1275,1292,1296,1309,1318,1330,1336,1336,1363
3,3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,...,1441,1455,1504,1520,1548,1577,1601,1613,1628,1660
4,4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,...,3776,3803,3881,3950,4036,4118,4191,4218,4234,4313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3224,3334,84056037,US,USA,840,56037.0,Sweetwater,Wyoming,US,41.659439,...,2451,2518,2544,2574,2649,2696,2702,2718,2762,2780
3225,3335,84056039,US,USA,840,56039.0,Teton,Wyoming,US,43.935225,...,1905,1930,1940,1958,1974,1990,1992,2002,2019,2033
3226,3336,84056041,US,USA,840,56041.0,Uinta,Wyoming,US,41.287818,...,1307,1327,1352,1363,1372,1399,1406,1416,1446,1460
3227,3338,84056043,US,USA,840,56043.0,Washakie,Wyoming,US,43.904516,...,639,644,659,662,665,690,695,699,713,720


In [252]:
!ls

2020.10.12.20211342.full.pdf      Project
Burgers_R10.zip                   burgers_data_R10.mat
COVID-19                          convdiff_2pi_n1500_t21_test
Dec                               data_prep_covid.ipynb
NavierStokes_V1e-4_N10000_T30.zip fourier_neural_operator
NavierStokes_V1e-5_N1200_T20.mat  updated_covid_us.csv
NavierStokes_V1e-5_N1200_T20.zip


In [7]:
cov_df2 = cov_df1.groupby('Province_State').sum()

In [8]:
cov_df2.head()

,Unnamed: 0,UID,code3,FIPS,Lat,Long_,1,2,3,4,...,327,328,329,330,331,332,333,334,335,336
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,2230,5628071489,56280,71489.0,2203.246784,-5809.578199,0,0,0,0,...,295631,297895,301533,305640,310335,315683,319904,322452,324832,329811
Alaska,2420,2436062340,24360,62340.0,1747.177765,-4293.070291,0,0,0,0,...,40933,41361,41643,42255,42641,43129,43462,43662,43819,44259
Arizona,1627,1260060208,12600,60208.0,505.138555,-1671.948482,0,0,0,0,...,408442,420237,424371,429219,435036,442671,448231,453597,461345,467204
Arkansas,11654,6300380625,63000,380625.0,2618.391704,-6932.548370,0,0,0,0,...,182606,183922,186117,188682,191757,194607,197289,198798,200302,202276
California,12994,4872351364,48720,351364.0,2194.949775,-7002.258461,0,0,0,0,...,1585378,1611493,1644742,1708559,1753974,1802195,1842557,1884033,1923887,1958508


In [9]:
cov_df3 = cov_df1.groupby('Province_State').count()

In [10]:
cov_df3.head()

,Unnamed: 0,UID,iso2,iso3,code3,FIPS,Admin2,Country_Region,Lat,Long_,...,327,328,329,330,331,332,333,334,335,336
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,67,67,67,67,67,67,67,67,67,67,...,67,67,67,67,67,67,67,67,67,67
Alaska,29,29,29,29,29,29,29,29,29,29,...,29,29,29,29,29,29,29,29,29,29
Arizona,15,15,15,15,15,15,15,15,15,15,...,15,15,15,15,15,15,15,15,15,15
Arkansas,75,75,75,75,75,75,75,75,75,75,...,75,75,75,75,75,75,75,75,75,75
California,58,58,58,58,58,58,58,58,58,58,...,58,58,58,58,58,58,58,58,58,58


In [11]:
cov_df2['Lat']/cov_df3['Lat']

Province_State
Alabama                 32.884280
Alaska                  60.247509
Arizona                 33.675904
Arkansas                34.911889
California              37.843962
Colorado                38.935474
Connecticut             41.613680
Delaware                39.109277
District of Columbia    38.904178
Florida                 28.940755
Georgia                 32.808532
Hawaii                  21.028283
Idaho                   44.263401
Illinois                39.843618
Indiana                 39.837353
Iowa                    42.029910
Kansas                  38.485379
Kentucky                37.622144
Louisiana               31.092427
Maine                   44.687688
Maryland                39.014921
Massachusetts           42.066277
Michigan                44.043620
Minnesota               45.552013
Mississippi             32.792727
Missouri                38.456765
Montana                 46.974691
Nebraska                41.321795
Nevada                  39.255997

In [12]:
cov_df_fin = cov_df2.copy()
cov_df_fin['Lat'] = cov_df2['Lat']/cov_df3['Lat']
cov_df_fin['Long_'] = cov_df2['Long_']/cov_df3['Long_']

In [13]:
cov_df_fin

,Unnamed: 0,UID,code3,FIPS,Lat,Long_,1,2,3,4,...,327,328,329,330,331,332,333,334,335,336
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,2230,5628071489,56280,71489.0,32.884280,-86.710122,0,0,0,0,...,295631,297895,301533,305640,310335,315683,319904,322452,324832,329811
Alaska,2420,2436062340,24360,62340.0,60.247509,-148.036907,0,0,0,0,...,40933,41361,41643,42255,42641,43129,43462,43662,43819,44259
Arizona,1627,1260060208,12600,60208.0,33.675904,-111.463232,0,0,0,0,...,408442,420237,424371,429219,435036,442671,448231,453597,461345,467204
Arkansas,11654,6300380625,63000,380625.0,34.911889,-92.433978,0,0,0,0,...,182606,183922,186117,188682,191757,194607,197289,198798,200302,202276
California,12994,4872351364,48720,351364.0,37.843962,-120.728594,0,0,0,0,...,1585378,1611493,1644742,1708559,1753974,1802195,1842557,1884033,1923887,1958508
Colorado,18356,5376515983,53760,515983.0,38.935474,-105.486240,0,0,0,0,...,288168,291079,293357,296691,300389,304082,306573,308865,311011,313527
Connecticut,2599,672072064,6720,72064.0,41.613680,-72.656830,0,0,0,0,...,145706,152848,154650,157068,159455,162120,162120,162120,166687,168287
Delaware,997,252030009,2520,30009.0,39.109277,-75.533237,0,0,0,0,...,45364,46359,47039,47824,48650,48990,50016,50496,50937,51501
District of Columbia,337,84011001,840,11001.0,38.904178,-77.016560,0,0,0,0,...,24874,25038,25339,25602,25830,26104,26342,26601,26740,26900


In [283]:
#t.pkl will be same. 
#x.pkl
x_list = [[]]
for index, row in cov_df_fin.iterrows():
    #print([row['Lat'], row['Long_']])
    x_list[0].append([row['Lat'], row['Long_']])
xp = np.asarray(x_list)
xp.shape

(1, 53, 2)

In [284]:
with open("covid_state_train_"+"x.pkl", "wb") as f:
    pkl.dump(xp, f, protocol=4)

In [285]:
#u.pkl -> daily cases
u_list = []
for i in range(1,181):
    u_list.append(cov_df_fin[i+1].values - cov_df_fin[i].values)
a,b = np.asarray(u_list).shape
print(a,b)
up_daily = np.asarray(u_list).reshape(1,a,b,1)
with open("covid_state_daily_train_"+"u.pkl", "wb") as f:
    pkl.dump(upt, f, protocol=4)

180 53


In [53]:
cov_df_fin_daily = cov_df_fin.copy()

In [61]:
#u.pkl test -> daily cases
u_list = []
maxm = 0
minm = 99999

for i in range(1,301):
    cov_df_fin_daily[str(i)] = cov_df_fin[str(i+1)].values - cov_df_fin[str(i)].values
    cov_df_fin_daily[str(i)] [cov_df_fin_daily[str(i)] < 0]= 0
    maxm = max(maxm,cov_df_fin_daily[str(i)].max())
    minm = min(minm,cov_df_fin_daily[str(i)].min())
    u_list.append(cov_df_fin[str(i+1)].values - cov_df_fin[str(i)].values)
a,b = np.asarray(u_list).shape
print(a,b)
up_daily = np.asarray(u_list).reshape(1,a,b,1)
# with open("covid_state_daily_test_"+"u.pkl", "wb") as f:
#     pkl.dump(upt, f, protocol=4)

<ipython-input-61-229e07bd11ae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cov_df_fin_daily[str(i)] [cov_df_fin_daily[str(i)] < 0]= 0
<ipython-input-61-229e07bd11ae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cov_df_fin_daily[str(i)] [cov_df_fin_daily[str(i)] < 0]= 0
<ipython-input-61-229e07bd11ae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cov_df_fin_daily[str(i)] [cov_df_fin_daily[str(i)] < 0]= 0
<ipython-input-61-2

<ipython-input-61-229e07bd11ae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cov_df_fin_daily[str(i)] [cov_df_fin_daily[str(i)] < 0]= 0
<ipython-input-61-229e07bd11ae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cov_df_fin_daily[str(i)] [cov_df_fin_daily[str(i)] < 0]= 0
<ipython-input-61-229e07bd11ae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cov_df_fin_daily[str(i)] [cov_df_fin_daily[str(i)] < 0]= 0
<ipython-input-61-2

300 53


<ipython-input-61-229e07bd11ae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cov_df_fin_daily[str(i)] [cov_df_fin_daily[str(i)] < 0]= 0
<ipython-input-61-229e07bd11ae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cov_df_fin_daily[str(i)] [cov_df_fin_daily[str(i)] < 0]= 0
<ipython-input-61-229e07bd11ae>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cov_df_fin_daily[str(i)] [cov_df_fin_daily[str(i)] < 0]= 0
<ipython-input-61-2

In [63]:
cov_df_fin[str(151)].values

array([ 29683,    856,  50127,  14582, 175243,  30333,  45439,  10662,
         9984,  93680,  58819,    777,   4004, 136049,  42061,  25491,
        12097,  13609,  49377,   2936,  63956, 101959,  63240,  32399,
        20641,  18057,    698,  17520,  12976,   5517, 168685,   9608,
       387272,  53009,   3251,  44261,  10038,   6750,  85590,   1473,
        14524,  23756,   6225,  33709, 109626,  17068,   1142,     73,
        57443,  28658,   2500,  24539,   1179])

In [64]:
cov_df_fin_daily[str(151)].values

array([ 439,   14, 2464,  408, 3414,  191,   43,   93,   36, 3493,  716,
         25,    0,  654,  362,  467,   39,  120,  393,   20,  350,   79,
        165,  439,    0,  403,   19,  105,  264,   25,  374,   94,  664,
       1120,   37,  547,  478,  187,  345,    9,    0,  905,   72,  537,
       3315,  394,   12,    3,  551,  198,   33,  280,   18])

In [65]:
maxm, minm

(33046, 0)

In [66]:
cov_df_fin_daily_norm = cov_df_fin_daily.copy()

In [67]:
maxm2 = 0
for i in range(1,301):
    cov_df_fin_daily_norm[str(i)] = cov_df_fin_daily[str(i)].values/maxm
    maxm2 = max(maxm2,cov_df_fin_daily_norm[str(i)].max())
maxm2

1.0

In [68]:
cov_df_fin_daily_norm

,Unnamed: 0,UID,code3,FIPS,Lat,Long_,1,2,3,4,...,327,328,329,330,331,332,333,334,335,336
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,2230,5628071489,56280,71489.0,32.884280,-86.710122,0.0,0.00000,0.0,0.000000,...,295631,297895,301533,305640,310335,315683,319904,322452,324832,329811
Alaska,2420,2436062340,24360,62340.0,60.247509,-148.036907,0.0,0.00000,0.0,0.000000,...,40933,41361,41643,42255,42641,43129,43462,43662,43819,44259
Arizona,1627,1260060208,12600,60208.0,33.675904,-111.463232,0.0,0.00000,0.0,0.000030,...,408442,420237,424371,429219,435036,442671,448231,453597,461345,467204
Arkansas,11654,6300380625,63000,380625.0,34.911889,-92.433978,0.0,0.00000,0.0,0.000000,...,182606,183922,186117,188682,191757,194607,197289,198798,200302,202276
California,12994,4872351364,48720,351364.0,37.843962,-120.728594,0.0,0.00000,0.0,0.000061,...,1585378,1611493,1644742,1708559,1753974,1802195,1842557,1884033,1923887,1958508
Colorado,18356,5376515983,53760,515983.0,38.935474,-105.486240,0.0,0.00000,0.0,0.000000,...,288168,291079,293357,296691,300389,304082,306573,308865,311011,313527
Connecticut,2599,672072064,6720,72064.0,41.613680,-72.656830,0.0,0.00000,0.0,0.000000,...,145706,152848,154650,157068,159455,162120,162120,162120,166687,168287
Delaware,997,252030009,2520,30009.0,39.109277,-75.533237,0.0,0.00000,0.0,0.000000,...,45364,46359,47039,47824,48650,48990,50016,50496,50937,51501
District of Columbia,337,84011001,840,11001.0,38.904178,-77.016560,0.0,0.00000,0.0,0.000000,...,24874,25038,25339,25602,25830,26104,26342,26601,26740,26900


In [69]:
#u.pkl train -> daily cases normalized 
u_list_train = []

for i in range(1,181):
    u_list_train.append(cov_df_fin_daily_norm[str(i)].values)
a,b = np.asarray(u_list_train).shape
print(a,b)
up_daily_norm_train = np.asarray(u_list_train).reshape(1,a,b,1)
with open("covid_state_daily_norm_train_"+"u.pkl", "wb") as f:
    pkl.dump(up_daily_norm_train, f, protocol=4)

180 53


In [70]:
#u.pkl test -> daily cases normalized 
u_list_test = []

for i in range(1,301):
    u_list_test.append(cov_df_fin_daily_norm[str(i)].values)
a,b = np.asarray(u_list_test).shape
print(a,b)
up_daily_norm_test = np.asarray(u_list_test).reshape(1,a,b,1)
with open("covid_state_daily_norm_test_"+"u.pkl", "wb") as f:
    pkl.dump(up_daily_norm_test, f, protocol=4)

300 53


In [71]:
lat_max = cov_df_fin_daily_norm['Lat'].max()
lat_min = cov_df_fin_daily_norm['Lat'].min()
long_max = cov_df_fin_daily_norm['Long_'].max()
long_min = cov_df_fin_daily_norm['Long_'].min()
lat_max, lat_min, long_max, long_min

(60.24750915137932, 18.239787564102564, -64.8963, -157.32323648)

,Unnamed: 0,UID,code3,FIPS,Lat,Long_,1,2,3,4,...,327,328,329,330,331,332,333,334,335,336
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,2230,5628071489,56280,71489.0,32.884280,-86.710122,0,0,0,0,...,295631,297895,301533,305640,310335,315683,319904,322452,324832,329811
Alaska,2420,2436062340,24360,62340.0,60.247509,-148.036907,0,0,0,0,...,40933,41361,41643,42255,42641,43129,43462,43662,43819,44259
Arizona,1627,1260060208,12600,60208.0,33.675904,-111.463232,0,0,0,0,...,408442,420237,424371,429219,435036,442671,448231,453597,461345,467204
Arkansas,11654,6300380625,63000,380625.0,34.911889,-92.433978,0,0,0,0,...,182606,183922,186117,188682,191757,194607,197289,198798,200302,202276
California,12994,4872351364,48720,351364.0,37.843962,-120.728594,0,0,0,0,...,1585378,1611493,1644742,1708559,1753974,1802195,1842557,1884033,1923887,1958508
Colorado,18356,5376515983,53760,515983.0,38.935474,-105.486240,0,0,0,0,...,288168,291079,293357,296691,300389,304082,306573,308865,311011,313527
Connecticut,2599,672072064,6720,72064.0,41.613680,-72.656830,0,0,0,0,...,145706,152848,154650,157068,159455,162120,162120,162120,166687,168287
Delaware,997,252030009,2520,30009.0,39.109277,-75.533237,0,0,0,0,...,45364,46359,47039,47824,48650,48990,50016,50496,50937,51501
District of Columbia,337,84011001,840,11001.0,38.904178,-77.016560,0,0,0,0,...,24874,25038,25339,25602,25830,26104,26342,26601,26740,26900


### Normalized latitude and longitude

In [75]:
x_list = [[]]
max_check = -999
for index, row in cov_df_fin_daily_norm.iterrows():
    lat_norm = (row['Lat'] - lat_min)/(lat_max- lat_min)
    long_norm = (row['Long_'] - long_min)/(long_max -long_min)
    max_check = max(max_check,long_norm)
    x_list[0].append([lat_norm, long_norm])
xp = np.asarray(x_list)
print(xp.shape, max_check)
with open("covid_state_norm_"+"x.pkl", "wb") as f:
    pkl.dump(xp, f, protocol=4)

(1, 53, 2) 1.0


### Normalized time

In [79]:
tp = (np.array(range(1,181))/180).reshape(1,180)
with open("covid_time_norm_train_"+"t.pkl", "wb") as f:
    pkl.dump(tp.reshape(1,180), f, protocol=4)

In [80]:
tpt = (np.array(range(1,301))/300).reshape(1,300)
with open("covid_time_norm_test_"+"t.pkl", "wb") as f:
    pkl.dump(tpt.reshape(1,300), f, protocol=4)

# Plotting GT and predicted count on MAP

In [90]:
cd /Users/abhay/GoogleDrive/BL/Project/Covid_GNN_PDE/code

/Users/abhay/GoogleDrive/BL/Project/Covid_GNN_PDE/code


In [91]:
ls

__pycache__/                          state_daily_norm_tmp_fig/
data_pos.csv                          state_daily_norm_tmp_fig_10k/
data_pos.txt                          test.py
data_pos_1.txt                        tmp_covid/
data_t.csv                            tmp_covid_daily/
data_t.txt                            tmp_covid_state_daily/
data_t_1.txt                          tmp_covid_state_daily_norm/
diff_state_daily_norm.txt             tmp_covid_state_daily_norm_10k/
diff_state_daily_norm_10k.txt         tmp_logs_covid/
diffs.png                             train.py
grid.png                              u_gt.csv
model_covid.pth                       u_gt.txt
model_covid_daily.pth                 u_gt_1.csv
model_covid_state_daily.pth           u_gt_1.txt
model_covid_state_daily_norm.pth      u_pred.csv
model_covid_state_daily_norm_10k.pth  u_pred.txt
pdegs/                                u_pred_1.csv
state_daily_norm_tmp.npy              u_pred_1.txt
state_daily_norm_tmp_1.npy  

In [109]:
states_norm = pd.read_csv('data_pos.csv')

In [110]:
states = states_norm.copy()

In [111]:
states.head()

,Lat,Long
0,0.349,0.764
1,1.000,0.100
2,0.367,0.496
3,0.397,0.702
4,0.467,0.396


In [112]:
lat_max, lat_min, long_max, long_min = (60.24750915137932, 18.239787564102564, -64.8963, -157.32323648)

In [113]:
lat_max, lat_min, long_max, long_min

(60.24750915137932, 18.239787564102564, -64.8963, -157.32323648)

In [114]:
states['Lat'] = states['Lat']*(lat_max-lat_min) + lat_min

In [117]:
states.head()

,Lat,Long
0,32.900482,0.764
1,60.247509,0.100
2,33.656621,0.496
3,34.916853,0.702
4,37.857394,0.396


In [128]:
states['Lat'].head()

0    32.900482
1    60.247509
2    33.656621
3    34.916853
4    37.857394
Name: Lat, dtype: float64

In [129]:
states['Long'].head()

0    -86.709057
1   -148.080543
2   -111.479476
3    -92.439527
4   -120.722170
Name: Long, dtype: float64

In [119]:
states['Long'] = states['Long']*(long_max-long_min) + long_min

In [121]:
states.head()

,Lat,Long
0,32.900482,-86.709057
1,60.247509,-148.080543
2,33.656621,-111.479476
3,34.916853,-92.439527
4,37.857394,-120.722170


In [123]:
lat = states['Lat'].values
lon = states['Long'].values

In [145]:
type(lat), lat.shape

(numpy.ndarray, (53,))

In [130]:
ls

__pycache__/                          state_daily_norm_tmp_fig/
data_pos.csv                          state_daily_norm_tmp_fig_10k/
data_pos.txt                          test.py
data_pos_1.txt                        tmp_covid/
data_t.csv                            tmp_covid_daily/
data_t.txt                            tmp_covid_state_daily/
data_t_1.txt                          tmp_covid_state_daily_norm/
diff_state_daily_norm.txt             tmp_covid_state_daily_norm_10k/
diff_state_daily_norm_10k.txt         tmp_logs_covid/
diffs.png                             train.py
grid.png                              u_gt.csv
model_covid.pth                       u_gt.txt
model_covid_daily.pth                 u_gt_1.csv
model_covid_state_daily.pth           u_gt_1.txt
model_covid_state_daily_norm.pth      u_pred.csv
model_covid_state_daily_norm_10k.pth  u_pred.txt
pdegs/                                u_pred_1.csv
state_daily_norm_tmp.npy              u_pred_1.txt
state_daily_norm_tmp_1.npy  

In [137]:
u_gt = pd.read_csv('u_gt.csv',  header=None)
print(len(u_gt))
u_gt.head()

300


,0,1,2,3,4,5,6,7,8,9,...,43,44,45,46,47,48,49,50,51,52
0,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.00003,0.0,0.000061,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.00000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [141]:
u_gt.iloc[2].values

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0.])

In [146]:
type(u_gt.iloc[2].values), u_gt.iloc[2].values.shape

(numpy.ndarray, (53,))

In [ ]:

population = cities['population_total'].values
area = cities['area_total_km2'].values

In [ ]:
# 1. Draw the map background
fig = plt.figure(figsize=(8, 8))
m = Basemap(projection='lcc', resolution='h', 
            lat_0=37.5, lon_0=-119,
            width=1E6, height=1.2E6)
m.shadedrelief()
m.drawcoastlines(color='gray')
m.drawcountries(color='gray')
m.drawstates(color='gray')



In [ ]:
# 2. scatter city data, with color reflecting population
# and size reflecting area
m.scatter(lon, lat, latlon=True,
          c=np.log10(population), s=area,
          cmap='Reds', alpha=0.5)

In [ ]:
# 3. create colorbar and legend
plt.colorbar(label=r'$\log_{10}({\rm population})$')
plt.clim(3, 7)

In [ ]:
# make legend with dummy points
for a in [100, 300, 500]:
    plt.scatter([], [], c='k', alpha=0.5, s=a,
                label=str(a) + ' km$^2$')
plt.legend(scatterpoints=1, frameon=False,
           labelspacing=1, loc='lower left');

In [148]:
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt

m = Basemap(projection='mill',
            llcrnrlat = -90,
            llcrnrlon = -180,
            urcrnrlat = 90,
            urcrnrlon = 180,
            resolution='l')

m.drawcoastlines()
m.drawcountries(linewidth=2)
##m.drawstates(color='b')
##m.drawcounties(color='darkred')
#m.fillcontinents()
#m.etopo()
m.bluemarble()

plt.title('Basemap Tutorial')
plt.show()

ModuleNotFoundError: No module named 'mpl_toolkits.basemap'